In [1]:
!pip install boto3

In [2]:
# Initial imports
import os
import io
import json
import numpy as np
import pandas as pd
import boto3

In [3]:
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = "spark-3.2.0"
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Luggage").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
file = "amazon_reviews_us_Luggage_v1_00.tsv.gz"

from pyspark.sql.types import IntegerType, DateType, StructType, StructField

spark.sparkContext.addFile(url)
df = spark.read.option("delimiter", "\t").option("header","True").option("InferSchema","True").csv(SparkFiles.get(file))

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [6]:
print(f"Number of rows: {df.count()}")
print(f"Number of distinct rows: {df.distinct().count()}")

Number of rows: 348657
Number of distinct rows: 348657


# Cleaning

In [7]:
# Drop duplicate rows - not necessary but included for consistency
df_distinct = df.distinct()
df_distinct.count()

348657

## Review_id_table

In [9]:
from pyspark.sql.functions import to_date
# Create review_id_table
review_id_table = df_distinct.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date"))
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3OHGQ6JOIBQ48|     128130|B003EJ050Y|     486561015| 2015-08-31|
|R1NQ9X24HV2U2E|   11836725|B005GUQ4CM|     317832287| 2015-08-31|
| RJI0Q00MCMKQH|   24710655|B00K6PGH7E|     931891804| 2015-08-31|
|R20SZBU9IS24TI|   36666557|B012U4W5TW|     173270557| 2015-08-30|
| RG5JVUZKMU1VM|   31021648|B00W9BSZO6|     967192081| 2015-08-29|
|R1336COI4RMITE|   31932003|B000M5G1TW|     758976344| 2015-08-29|
|R33EO3BLNVAXYR|   23681709|B005IFCZ78|     143380281| 2015-08-29|
|R2W99FE9TTAOF0|   13793484|B00W4TAAYG|     456332879| 2015-08-29|
| R9CKZLJKCLSPD|   33805050|B005AIISEC|     128696805| 2015-08-28|
| RZMOB0WJAM6AD|   38135040|B00N456A5C|     613530019| 2015-08-28|
| RF9OIRURI3VA8|   50753128|B00KGDZJAM|     781566654| 2015-08-28|
|R3FMRTNUAG0Q9A|   45053111|B003KR1K4K|     689597691| 2015-08

In [10]:
# Check data types
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

## products

In [11]:
products = df_distinct.select("product_id", "product_title")
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003EJ050Y|Olympia Luggage S...|
|B005GUQ4CM|Everest Deluxe To...|
|B00K6PGH7E|Visconti Visconti...|
|B012U4W5TW|Shirts By Sarah T...|
|B00W9BSZO6|Travelon Anti-The...|
|B000M5G1TW|DESIGN GO TRAVEL ...|
|B005IFCZ78|Everest Luggage B...|
|B00W4TAAYG|Polare Canvas Cow...|
|B005AIISEC|Travelon Compress...|
|B00N456A5C|Disney Pixar Cars...|
|B00KGDZJAM|Tumi Prism Leathe...|
|B003KR1K4K|Samsonite Tote-A-...|
|B0053518QM|   Everest Sling Bag|
|B003FA2APK|Leighton Black UV...|
|B00HT8BBHM|Samantha Brown Fi...|
|B004CI8CQA|Samsonite Bonded ...|
|B00NWUH760|Rockland Stage Co...|
|B00RYTYAQ0|IGOGEER Money Bel...|
|B00NGGEMJ0|Aibag Multi-Color...|
|B00VZ67AFG|Bellaza Quilted S...|
+----------+--------------------+
only showing top 20 rows



In [12]:
products.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [13]:
# Drop duplicates to keep only one review per ID
# Loses data, but is needed for upload to table with specified schema
products_dist = products.dropDuplicates(['product_id'])

In [14]:
products_dist.count()

64589

## customers

In [15]:
customers = df_distinct.groupby("customer_id").count()
customers = customers.withColumnRenamed("count","customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   14750529|             2|
|     264914|             1|
|   34326854|             1|
|   41688270|             4|
|   18800155|             2|
|   15495404|             1|
|   29705442|             1|
|   14777327|             1|
|   18255735|             2|
|   35575504|             1|
|   15123832|             1|
|   13216996|             8|
|   38715395|             1|
|   27632536|             1|
|   47143360|             1|
|    1721110|             1|
|   23864483|             1|
|   38756115|             1|
|   52110649|             1|
|     481231|             1|
+-----------+--------------+
only showing top 20 rows



In [16]:
customers.dtypes

[('customer_id', 'int'), ('customer_count', 'bigint')]

## vine_table

In [17]:
vine_table = df_distinct.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3OHGQ6JOIBQ48|          5|            0|          0|   N|
|R1NQ9X24HV2U2E|          5|            0|          0|   N|
| RJI0Q00MCMKQH|          5|            0|          0|   N|
|R20SZBU9IS24TI|          5|            1|          1|   N|
| RG5JVUZKMU1VM|          3|            2|          3|   N|
|R1336COI4RMITE|          5|            0|          0|   N|
|R33EO3BLNVAXYR|          4|            0|          0|   N|
|R2W99FE9TTAOF0|          5|            0|          0|   N|
| R9CKZLJKCLSPD|          4|            0|          0|   N|
| RZMOB0WJAM6AD|          4|            0|          0|   N|
| RF9OIRURI3VA8|          5|            0|          0|   N|
|R3FMRTNUAG0Q9A|          5|            1|          1|   N|
|R23FABLBZ3E67H|          4|            0|          0|   N|
|R19LVAC1TJHFUD|          5|            

In [18]:
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

# Database connection

In [19]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-26 20:47:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.72MB/s    in 0.5s    

2021-11-26 20:47:08 (1.72 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [20]:
mode = "append"
jdbc_url="jdbc:postgresql://<instance>.rds.amazonaws.com:5432/bigdatachallenge"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [22]:

products_dist.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [23]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)